In [19]:
import pandas as pd
import numpy as np
from plotly import graph_objects as go

In [2]:
df = pd.read_csv('events.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 885129 entries, 0 to 885128
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   event_time     885129 non-null  object 
 1   event_type     885129 non-null  object 
 2   product_id     885129 non-null  int64  
 3   category_id    885129 non-null  int64  
 4   category_code  648910 non-null  object 
 5   brand          672765 non-null  object 
 6   price          885129 non-null  float64
 7   user_id        885129 non-null  int64  
 8   user_session   884964 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 60.8+ MB


In [27]:
df.describe()

,product_id,category_id,price,user_id
count,8.851290e+05,8.851290e+05,885129.000000,8.851290e+05
mean,1.906621e+06,2.144423e+18,146.328713,1.515916e+18
std,1.458708e+06,6.165105e+14,296.807683,3.747287e+07
min,1.020000e+02,2.144416e+18,0.220000,1.515916e+18
25%,6.988030e+05,2.144416e+18,26.460000,1.515916e+18
50%,1.452883e+06,2.144416e+18,65.710000,1.515916e+18
75%,3.721194e+06,2.144416e+18,190.490000,1.515916e+18
max,4.183880e+06,2.227847e+18,64771.060000,1.515916e+18


In [4]:
df.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2020-09-24 11:57:06 UTC,view,1996170,2144415922528452715,electronics.telephone,NaN,31.90,1515915625519388267,LJuJVLEjPT
1,2020-09-24 11:57:26 UTC,view,139905,2144415926932472027,computers.components.cooler,zalman,17.16,1515915625519380411,tdicluNnRY
2,2020-09-24 11:57:27 UTC,view,215454,2144415927158964449,NaN,NaN,9.81,1515915625513238515,4TMArHtXQy
3,2020-09-24 11:57:33 UTC,view,635807,2144415923107266682,computers.peripherals.printer,pantum,113.81,1515915625519014356,aGFYrNgC08
4,2020-09-24 11:57:36 UTC,view,3658723,2144415921169498184,NaN,cameronsino,15.87,1515915625510743344,aa4mmk0kwQ


In [5]:
set(df['event_type'])

{'cart', 'purchase', 'view'}

In [6]:
df['datetime'] = pd.to_datetime(df['event_time'])

In [7]:
df.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,datetime
0,2020-09-24 11:57:06 UTC,view,1996170,2144415922528452715,electronics.telephone,NaN,31.90,1515915625519388267,LJuJVLEjPT,2020-09-24 11:57:06+00:00
1,2020-09-24 11:57:26 UTC,view,139905,2144415926932472027,computers.components.cooler,zalman,17.16,1515915625519380411,tdicluNnRY,2020-09-24 11:57:26+00:00
2,2020-09-24 11:57:27 UTC,view,215454,2144415927158964449,NaN,NaN,9.81,1515915625513238515,4TMArHtXQy,2020-09-24 11:57:27+00:00
3,2020-09-24 11:57:33 UTC,view,635807,2144415923107266682,computers.peripherals.printer,pantum,113.81,1515915625519014356,aGFYrNgC08,2020-09-24 11:57:33+00:00
4,2020-09-24 11:57:36 UTC,view,3658723,2144415921169498184,NaN,cameronsino,15.87,1515915625510743344,aa4mmk0kwQ,2020-09-24 11:57:36+00:00


In [ ]:
# 이탈율 분석

In [25]:
view_cnt_per_session = pd.pivot_table(df, index=['user_session', 'user_id'], values= ['event_time', 'datetime'], aggfunc={'event_time': 'count', 'datetime':min}).reset_index()
view_cnt_per_session['month'] = view_cnt_per_session['datetime'].dt.strftime('%Y-%m')

In [30]:
view_cnt_per_session['bounced'] = view_cnt_per_session.event_time == 1
view_cnt_per_session['bounced'] = view_cnt_per_session['bounced'].astype(int)

In [92]:
bounced = pd.pivot_table(view_cnt_per_session, index='month', columns='bounced', aggfunc={'count'})['datetime']

In [101]:
bounced = bounced.reset_index(col_level=1)
bounced = bounced.droplevel(axis=1,level=0)
bounced = bounced.rename(columns={0:'bounced', 1:'not_bounced'})

In [155]:
bounced['bounced_rate'] = bounced['bounced'] / ( bounced['bounced'] + bounced['not_bounced'] ) * 100

In [156]:
bounced

bounced,level_0,index,month,bounced,not_bounced,bounced_rate
0,0,0,2020-09,4715,12276,27.749985
1,1,1,2020-10,25877,69569,27.111665
2,2,2,2020-11,29912,77860,27.754890
3,3,3,2020-12,24724,60149,29.130583
4,4,4,2021-01,29637,67909,30.382589
5,5,5,2021-02,25823,62182,29.342651


In [ ]:
# 이탈률이 증가하는 추세임.
# 유저들이 어떻게 유입됐는지 알 수 있으면 이탈 요인 분석할 수 있을 듯

In [ ]:
# 코호트별 이탈률 분석

In [ ]:
# 고객 세그먼트별 이탈률 분석

In [ ]:
# RFM 이외에 브랜드 제품 구매 비율 변수를 추가해서 고객 분류

In [ ]:
# 어떤 제품을 본 유저들이 이탈했는지 추가 분석 : 특정 제품군을 본 유저들이 많이 이탈했다면 해당 제품군은 이 사이트 이용자에게 별로 흥미를 주지 못하는 것으로 해석될 수 있음.

In [119]:
view_cnt_per_session = pd.merge(df[['user_session', 'datetime', 'category_code', 'brand']], view_cnt_per_session, on=['user_session', 'datetime'], how='right')

In [143]:
bounced_cat = pd.pivot_table(view_cnt_per_session, index=['month', 'category_code'], columns='bounced', aggfunc={'count'})['datetime'].fillna(0)

In [144]:
bounced_cat = bounced_cat.reset_index(col_level=1)
bounced_cat = bounced_cat.droplevel(axis=1,level=0)
bounced_cat = bounced_cat.rename(columns={0:'bounced', 1:'not_bounced'})

In [158]:
bounced_cat['bounced_rate'] = bounced_cat['bounced'] / ( bounced_cat['bounced'] + bounced_cat['not_bounced'] )

In [163]:
bounced_cat_tot = bounced_cat.groupby(by='category_code').sum()[['bounced', 'not_bounced']]

In [167]:
bounced_cat_tot['bounced_rate'] = bounced_cat_tot['bounced'] / ( bounced_cat_tot['bounced'] + bounced_cat_tot['not_bounced'] ) * 100

In [169]:
#이탈률이 높은 제품군 상위 50위
bounced_cat_tot.sort_values(by=['bounced_rate'], ascending=False)[:50]

bounced,bounced,not_bounced,bounced_rate
category_code,,,
furniture.bedroom.blanket,8.0,9.0,47.058824
computers.components.videocards,19525.0,23902.0,44.960508
auto.accessories.parktronic,371.0,515.0,41.873589
computers.peripherals.camera,988.0,1527.0,39.284294
electronics.audio.dictaphone,59.0,95.0,38.311688
computers.peripherals.scanner,387.0,652.0,37.247353
country_yard.cultivator,489.0,829.0,37.101669
computers.components.motherboard,4117.0,7026.0,36.946962
auto.accessories.player,3103.0,5562.0,35.810733


In [171]:
#이탈률이 낮은 제품군 상위 50위
bounced_cat_tot.sort_values(by=['bounced_rate'], ascending=True)[:50]

bounced,bounced,not_bounced,bounced_rate
category_code,,,
appliances.kitchen.washer,4.0,51.0,7.272727
auto.accessories.light,3.0,23.0,11.538462
country_yard.watering,4.0,30.0,11.764706
construction.tools.generator,18.0,126.0,12.500000
computers.gaming,23.0,145.0,13.690476
furniture.living_room.chair,5.0,31.0,13.888889
appliances.personal.scales,61.0,376.0,13.958810
kids.toys,19.0,107.0,15.079365
appliances.kitchen.refrigerators,2.0,11.0,15.384615
